# Dataset editing

In [2]:
import skimage.transform
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
hira = np.load("hiragana.npz")['arr_0'].reshape([-1, 127, 128]).astype(np.float32)

hira = hira/np.max(hira)

In [4]:
# 71 characters, 160 writers, transform image to 48*48
train_images = np.zeros([71 * 160, 48, 48], dtype=np.float32)

for i in range(71 * 160):
    train_images[i] = skimage.transform.resize(hira[i], (48, 48))

arr = np.arange(71)
train_labels = np.repeat(arr, 160) # create labels

# split to train and test
train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.2)

In [5]:
np.savez_compressed("hiragana_train_images.npz", train_images)
np.savez_compressed("hiragana_train_labels.npz", train_labels)
np.savez_compressed("hiragana_test_images.npz", test_images)
np.savez_compressed("hiragana_test_labels.npz", test_labels)

# Actual training

In [6]:
from tensorflow import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

2024-01-31 12:52:53.554870: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 12:52:53.886189: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 12:52:53.886319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 12:52:53.929925: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 12:52:54.018956: I tensorflow/core/platform/cpu_feature_guar

In [7]:
train_images = np.load("hiragana_train_images.npz")['arr_0']
train_labels = np.load("hiragana_train_labels.npz")['arr_0']
test_images = np.load("hiragana_test_images.npz")['arr_0']
test_labels = np.load("hiragana_test_labels.npz")['arr_0']

In [8]:
if K.image_data_format() == "channels_first":
  train_images = train_images.reshape(train_images.shape[0], 1,48,48)
  test_images2 = test_images.reshape(test_images.shape[0], 1,48,48)
  shape = (1,48,48)
else:
  train_images = train_images.reshape(train_images.shape[0], 48, 48, 1)
  test_images2 = test_images.reshape(test_images.shape[0], 48, 48, 1)
  shape = (48,48,1)

datagen = ImageDataGenerator(rotation_range=15,zoom_range=0.2)
datagen.fit(train_images)

model = keras.Sequential([
  keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=shape),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Conv2D(64, (3,3), activation='relu'),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Conv2D(64, (3,3), activation='relu'),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Flatten(),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(1024, activation='relu'),
  keras.layers.Dense(71, activation="softmax")
])

model.summary()

model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 64)          36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 4, 4, 64)          0

2024-01-31 12:53:45.383580: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 12:53:45.497878: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 12:53:45.498372: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [9]:
model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

model.fit_generator(datagen.flow(train_images,train_labels,shuffle=True),epochs=30,validation_data=(test_images2,test_labels),callbacks = [keras.callbacks.EarlyStopping(patience=8,verbose=1,restore_best_weights=True),keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)])

test_loss, test_acc = model.evaluate(test_images2, test_labels)
print("Test Accuracy: ", test_acc)

Epoch 1/30


/tmp/ipykernel_32367/29902955.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(train_images,train_labels,shuffle=True),epochs=30,validation_data=(test_images2,test_labels),callbacks = [keras.callbacks.EarlyStopping(patience=8,verbose=1,restore_best_weights=True),keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)])
2024-01-31 12:54:00.175809: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-31 12:54:00.311249: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-31 12:54:00.633770: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-31 12:54:00.992002: I external/local_xla/xla/service/service.cc:168] XLA service 0x76bf640128e0 initialized for 

284/284 [==============================] - 3s 5ms/step - loss: 2.3023 - accuracy: 0.3996 - val_loss: 0.6590 - val_accuracy: 0.8099 - lr: 0.0010
Epoch 2/30
284/284 [==============================] - 1s 4ms/step - loss: 0.9101 - accuracy: 0.7261 - val_loss: 0.3407 - val_accuracy: 0.9005 - lr: 0.0010
Epoch 3/30
284/284 [==============================] - 1s 4ms/step - loss: 0.6050 - accuracy: 0.8107 - val_loss: 0.2411 - val_accuracy: 0.9115 - lr: 0.0010
Epoch 4/30
284/284 [==============================] - 1s 4ms/step - loss: 0.4691 - accuracy: 0.8461 - val_loss: 0.2907 - val_accuracy: 0.9027 - lr: 0.0010
Epoch 5/30
284/284 [==============================] - 1s 4ms/step - loss: 0.3779 - accuracy: 0.8778 - val_loss: 0.1353 - val_accuracy: 0.9489 - lr: 0.0010
Epoch 6/30
284/284 [==============================] - 1s 4ms/step - loss: 0.3215 - accuracy: 0.8943 - val_loss: 0.1027 - val_accuracy: 0.9670 - lr: 0.0010
Epoch 7/30
284/284 [==============================] - 1s 4ms/step - loss: 0.2472 

In [11]:
model.save("hiragana.keras")